In [17]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [18]:
! pip3 install torch torchvision torchaudio

In [19]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    pipeline,
    logging,
)
from transformers import TrainingArguments
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format
from trl import SFTConfig

In [20]:
from huggingface_hub import login
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()

hf_token = ""
login(token = hf_token)

In [14]:

! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/Users/betasys.ai/Desktop/ll

In [6]:
wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on Medical Dataset', 
    job_type="training", 
    anonymous="allow"`
    +2
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dbarde75 (dbarde75-betasys-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [21]:
#base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
#base_model =  "kingabzpro/llama-3-8b-chat-doctor"#"mathewhe/Llama-3.1-8B-Chat"# "baffo32/decapoda-research-llama-7B-hf"#"decapoda-research/llama-7b-hf"
base_model =  "gemma/transformers/1.1-2b-it/1"
dataset_name = "ruslanmv/ai-medical-chatbot"
new_model = "gemma-2b-chat"

In [25]:
# Set torch dtype and attention implementation
# if torch.cuda.get_device_capability()[0] >= 8:
#     !pip install -qqq flash-attn
#     torch_dtype = torch.bfloat16
#     attn_implementation = "flash_attention_2"
# else:
#     torch_dtype = torch.float16
#     attn_implementation = "eager"

import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch_dtype = torch.float16  # or torch.float32 if needed
    attn_implementation = "eager"  # flash_attention_2 is not supported on MPS
else:
    device = torch.device("cpu")
    torch_dtype = torch.float32
    attn_implementation = "eager"

device


device(type='mps')

In [26]:
#base_model =  "kingabzpro/llama-3-8b-chat-doctor"
# base_model =  "/kaggle/input/gemma/transformers/1.1-2b-it/1"
base_model = "google/gemma-3-1b-it"

In [27]:
# QLoRA config
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch_dtype,
#     bnb_4bit_use_double_quant=True,
# )

# # Load model
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=bnb_config,
#     device_map="auto",
#     attn_implementation=attn_implementation
# )

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)





if torch.cuda.is_available():
    # CUDA path with quantization
    torch_dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
    attn_implementation = "flash_attention_2" if torch_dtype == torch.bfloat16 else "eager"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        device_map="auto",
        attn_implementation=attn_implementation
    )

else:
    # MPS or CPU fallback (Mac)
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
    torch_dtype = torch.float32
    attn_implementation = "eager"

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch_dtype,
        device_map={"": device},  # forces loading to MPS/CPU
        attn_implementation=attn_implementation
    )

# Load tokenizer (works the same on all platforms)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

In [28]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
#model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [29]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")

dataset = dataset.shuffle(seed=65).select(range(50000)) # Only use 1000 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= 4,
)

type(dataset)


datasets.arrow_dataset.Dataset

In [13]:
dataset[0]

{'Description': 'Can blood pressure medication be stopped to check improvement in bp levels?',
 'Patient': "I'm 35, BP 150/100 without medicine, never smoke, drink, BMIMy hdl:45, LDL:107, Total Cholesterol:168, Triglyceride:95, Blood Sugar(Fasting):83 (all are without medicine) My question is, should I stop this medicine to see the afffect. it is worth to mention that recently I've increased my physical activity,",
 'Doctor': 'Hello,Thanks for writing to Health Care Magic, I am Dr Asad Riaz, I have closely read your question and I understand your concerns, I will hereby guide you regarding your health related problem.BP is one major risk factor for major complication like MI or stroke...if pt have high BP as u r having ..1st we need to do life style modification to see whteher it work to de BP or not in whch i advice pt to lower salt intake,drinkng dec weight n exercise for a period of 6 month n still if pt have high BP then need to add med to control it so we prevent major comlication

In [25]:
dataset['text'][3]

'<bos><start_of_turn>user\nFell on sidewalk face first about 8 hrs ago. Swollen, cut lip bruised and cut knee, and hurt pride initially. Now have muscle and shoulder pain, stiff jaw(think this is from the really swollen lip),pain in wrist, and headache. I assume this is all normal but are there specific things I should look for or will I just be in pain for a while given the hard fall?<end_of_turn>\n<start_of_turn>model\nHello and welcome to HCM,The injuries caused on various body parts have to be managed.The cut and swollen lip has to be managed by sterile dressing.The body pains, pain on injured site and jaw pain should be managed by pain killer and muscle relaxant.I suggest you to consult your primary healthcare provider for clinical assessment.In case there is evidence of infection in any of the injured sites, a course of antibiotics may have to be started to control the infection.Thanks and take careDr Shailja P Wahal<end_of_turn>\n'

In [30]:
dataset = dataset.train_test_split(test_size=0.1)

In [31]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    #evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)


In [ ]:
import transformers
print(transformers.__version__)
help(TrainingArguments)


In [32]:
#Hyperparamter
# training_arguments = TrainingArguments(
#     output_dir=new_model,
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     gradient_accumulation_steps=2,
#     optim= "adamw_8bit",#"paged_adamw_32bit",
#     num_train_epochs=1,
#     eval_strategy ="steps",
#     eval_steps=100,  # Changed from 0.2 to 100
#     logging_steps=1,
#     warmup_steps=10,
#     logging_strategy="steps",
#     learning_rate=2e-4,
#     fp16=False,
#     no_cuda=True,      # Changed from False to True
#     bf16=False,
#     group_by_length=True,
#     report_to=[],
#     save_total_limit=2,
#     save_steps=500,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=True,
#     disable_tqdm=True
# )


training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",           # Disable saving (can conflict on M1)
    fp16=False,                   # Don't use mixed precision
    no_cuda=True,                 # Force not to use CUDA
    report_to=[],                 # Prevent HuggingFace Hub or WandB logging
    disable_tqdm=True             # For cleaner notebook output
)

/Users/betasys.ai/Desktop/llm-finetune/.venv/lib/python3.12/site-packages/transformers/training_args.py:1577: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [33]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    args=training_arguments
)

Truncating eval dataset: 100%|██████████| 5000/5000 [00:00<00:00, 855456.66 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [37]:
# Set environment variable to help with memory fragmentation
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:

# Train the model
trainer.train()

In [34]:
trainer.train()

{'loss': 4.764, 'grad_norm': 6.100841045379639, 'learning_rate': 4.999e-05, 'num_tokens': 2608.0, 'mean_token_accuracy': 0.28846983164548873, 'epoch': 0.00022222222222222223}
{'loss': 4.0883, 'grad_norm': 3.990191698074341, 'learning_rate': 4.9978888888888895e-05, 'num_tokens': 4416.0, 'mean_token_accuracy': 0.32465327680110934, 'epoch': 0.00044444444444444447}
{'loss': 3.4966, 'grad_norm': 4.589986801147461, 'learning_rate': 4.996777777777778e-05, 'num_tokens': 7071.0, 'mean_token_accuracy': 0.35612346827983854, 'epoch': 0.0006666666666666666}
{'loss': 3.4433, 'grad_norm': 4.498806476593018, 'learning_rate': 4.995666666666667e-05, 'num_tokens': 9562.0, 'mean_token_accuracy': 0.358892422914505, 'epoch': 0.0008888888888888889}
{'loss': 3.3234, 'grad_norm': 3.611513614654541, 'learning_rate': 4.9945555555555556e-05, 'num_tokens': 11723.0, 'mean_token_accuracy': 0.3869830995798111, 'epoch': 0.0011111111111111111}
{'loss': 3.1971, 'grad_norm': 4.646134376525879, 'learning_rate': 4.99344444

TrainOutput(global_step=45000, training_loss=2.573095331001282, metrics={'train_runtime': 67728.5314, 'train_samples_per_second': 0.664, 'train_steps_per_second': 0.664, 'total_flos': 4.432686583360435e+16, 'train_loss': 2.573095331001282})

In [35]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
# wandb.finish()
model.config.use_cache = True

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

In [21]:
messages = [{"role": "user", "content": "tajmahal"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[0])

user
tajmahal
model
Hi, I have gone through your query and understand your concern. Taj Mahal is a historical monument and a symbol of love. It is a UNESCO World Heritage Site. It is a symbol of love and devotion. It is a symbol of the rich history of India. It is a symbol of the love between the king and his queen. It is a symbol of the beauty of India. It is a symbol of the power of love. It is a symbol of the unity of India. It is a symbol of the unity of the world. It is a symbol of the peace of the world. It is a symbol of the hope of the world. It is a symbol of the future of


In [ ]:
messages = [{"role": "user", "content": "Hello doctor, I always feel weak, can you help me with that?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])